In [ ]:
import pickle
import os
md_path = '/hybedata/Images/Zach/ExVivo/Predigest_WholeMount_Third_2019Mar22/'
spotcalls = pickle.load(open(os.path.join(md_path,'results','spotcalls.pkl'),'rb'))
# from metadata import Metadata
# md = Metadata(md_path)

In [ ]:
from metadata import Metadata

In [ ]:
len(spotcalls)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
hist = np.histogram2d(spotcalls.CoordX,spotcalls.CoordY,bins=100)
plt.imshow(hist[0],cmap='inferno')
plt.show()

In [ ]:
%matplotlib inline 
from matplotlib import pyplot as plt
import matplotlib as mpl
from skimage.transform import resize
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
from sklearn.preprocessing import normalize
from scipy import ndimage
from skimage import morphology,filters
import pywt
import numpy as np
from scipy.stats import gaussian_kde
x_offset = int(2048*0.109)
y_offset = 0
x_min = -2000
x_max = -1100
y_min = 1300
y_max = 1400
# temp_df = spotcalls[(spotcalls.ave>1000)&
#                     (spotcalls.CoordY>(y_min-y_offset))&
#                     (spotcalls.CoordY<(y_max-y_offset))&
#                     (spotcalls.CoordX>(x_min-x_offset))&
#                     (spotcalls.CoordX<(x_max-x_offset))]
img_dict = {}
img_x_dict = {}
img_y_dict = {}
wavelet = 'db10'
keep_list=[4,5,6]
scale = int(2048*0.109)
X = []
Y = []

for pos in md.image_table.Position.unique():
    x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
    if not (x>x_min)&(x<x_max)&(y>y_min)&(y<y_max):
        continue
    X.append(x)
    Y.append(y)
    img_x_dict[pos]=x
    img_y_dict[pos]=y
    im = np.flipud(md.stkread(Position=pos,
                              Channel='DeepBlue',
                              acq='nuc_highWellCapacity_6',
                              Zindex=10)[:,:,0])
    thresh = np.percentile(im.ravel(),99)
    im[im>thresh]=thresh
    coeffs = pywt.wavedec2(im,wavelet)
    for i in range(0,len(coeffs)+1):
        if i in keep_list:
            continue
        coeffs[-i] = tuple([np.zeros_like(v) for v in coeffs[-i]])
    Result = pywt.waverec2(coeffs,wavelet)
    thresh = np.percentile(Result.ravel(),99.9)
    Result[Result>thresh]=thresh
    Result[im<2**8.5]=np.nan
    if (x>-2500)&(x<-500)&(y>800)&(y<2000):
        im_bin = im>500
    else:
        im_bin = im>2000
    im_bin = ndimage.binary_dilation(im_bin,iterations=5)
    im_bin = morphology.remove_small_holes(im_bin, 2000)
    im_bin = morphology.remove_small_objects(im_bin, 10000)
    im_bin = morphology.erosion(im_bin)
    Result[im_bin==False]=np.nan
    Result = Result-np.nanpercentile(Result.ravel(),1)
    Result = Result/np.nanpercentile(Result.ravel(),99)
    img_dict[pos]=Result

temp_mask = []
poses = img_dict.keys()
for pos in spotcalls.posname:
    if pos in poses:
        temp_mask.append(True)
    else:
        temp_mask.append(False)
temp_df = spotcalls[(spotcalls.ave>1000)&temp_mask]


In [ ]:
for pos in img_dict.keys():
    print(pos, img_x_dict[pos],img_y_dict[pos])

In [ ]:
Stitched = np.concatenate((img_dict['Pos62'],img_dict['Pos77'],img_dict['Pos92'],img_dict['Pos105']),axis=1)
plt.imshow(Stitched,cmap='Greys')

In [ ]:
X = []
for pos,x in img_x_dict.items():
    X.append(x)
Y = []
for pos,y in img_y_dict.items():
    Y.append(y)

In [ ]:
% matplotlib inline
norm = mpl.colors.Normalize(vmin=0,vmax=1)
BIGGER_SIZE = 12
plt.rc('font', size=BIGGER_SIZE, family='Arial') # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
temp_df = spotcalls[(spotcalls.ave>1000)&temp_mask]
x_offset = int(2048*0.109)
y_offset = 0
for gene in temp_df.gene.unique():
    temp = temp_df[(temp_df.gene==gene)]
    if len(temp)<100:
        continue
    fig = plt.figure(figsize=(4,1))
    ax = fig.add_subplot(111) 
    plt.imshow(Stitched,cmap='Greys')
    x = (temp.CoordX+(x_offset-np.min(X)))/0.109
    y = (temp.CoordY+(y_offset-np.min(Y)))/0.109
    z = temp.z
    plt.scatter(x,y,c=z,s=1)
    plt.xlim([0,2048*4])
    plt.ylim([0,2048])
    plt.title(gene)
    plt.show()

In [ ]:
% matplotlib inline
norm = mpl.colors.Normalize(vmin=0,vmax=1)
BIGGER_SIZE = 12
plt.rc('font', size=BIGGER_SIZE, family='Arial') # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
temp_df = spotcalls[(spotcalls.ave>1000)&temp_mask]
x_offset = int(2048*0.109)
y_offset = 0
for gene in temp_df.gene.unique():
    temp = temp_df[(temp_df.gene==gene)]
    if len(temp)<500:
        continue
    fig = plt.figure(figsize=(8,2))
    ax = fig.add_subplot(111) 
    plt.imshow(np.flipud(Stitched),cmap='Greys',norm=norm)
    x = (temp.CoordX+(x_offset-np.min(X)))/0.109
    y = (temp.CoordY+(y_offset-np.min(Y)))/0.109
    #z = temp.z
    plt.scatter(x,y,c='r',s=5,alpha=0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.ylabel(gene)
    plt.xlim([0,2048*4])
    plt.ylim([0,2048])
    #plt.axis('off')
    plt.show()

In [ ]:
% matplotlib inline
norm = mpl.colors.Normalize(vmin=0,vmax=1)
BIGGER_SIZE = 12
plt.rc('font', size=BIGGER_SIZE, family='Arial') # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
temp_df = spotcalls[(spotcalls.ave>1000)&temp_mask]
x_offset = int(2048*0.109)
y_offset = 0
gene_names = ['Tnfr21','Casp9','Lrp5']
gene_list = ['Tnfrsf21','Casp9','Lrp5']
colors = ['b','r','g']
fig = plt.figure(figsize=(3,3),dpi=4000)
ax = fig.add_subplot(414) 
x_axis = np.linspace(-50,750,num=30)
for i,gene in enumerate(gene_list):
    temp = temp_df[(temp_df.gene==gene)]
    x = (temp.CoordX+(x_offset-np.min(X)))/0.109
    AB = np.histogram(x,bins=30,range=[1000,(2048*4)-400])
    A = AB[0]
    B = AB[1][0:-1]
    plt.plot(x_axis,A/np.max(A),c=colors[i],label=gene)
plt.xlim([800,(2048*4)-600])
#plt.legend(loc='upper right',ncol=3)
plt.xlabel('Wound Distance ('+'\u03BC'+'m)',labelpad=-1)
#plt.gcf().subplots_adjust(bottom=0.25)
ax.set_xticks([0,250,500,750])
ax.set_yticks([0,1])
ax.yaxis.tick_right()
ax.set_xlim(-50,750)

for i,gene in enumerate(gene_list):
    temp = temp_df[(temp_df.gene==gene)]
    if len(temp)<500:
        continue
    loc = int('41'+str(i+1))
    ax1 = fig.add_subplot(loc) 
    plt.imshow(np.flipud(Stitched[:,0:-400]),cmap='Greys',norm=norm, aspect="auto")
    x = (temp.CoordX+(x_offset-np.min(X)))/0.109
    y = (temp.CoordY+(y_offset-np.min(Y)))/0.109
    z = temp.z
    plt.scatter(x,y,c=colors[i],s=1,alpha=0.5)
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.yaxis.set_label_position('right')
    plt.ylabel(gene_names[i])
    ax1.set_xlim(1000,(2048*4)-400)
    ax1.set_ylim(0,2048)
#     plt.xlim([1000,(2048*4)-400])
#     plt.ylim([0,2048])
    #plt.axis('off')
    
plt.show()
fig.savefig('/home/zach/Desktop/Wounded_Cornea_spots_zoom')

In [ ]:
X = np.linspace(-50,750,num=30)
X

In [ ]:
gene_hist_dict = {}
colors = ['b','r','g']
for i,gene in enumerate(gene_list):
    temp = temp_df[(temp_df.gene==gene)]
    x = (temp.CoordX+(x_offset-np.min(X)))/0.109
    AB = np.histogram(x,bins=30,range=[1000,(2048*4)-400])
    A = AB[0]
    B = AB[1][0:-1]
    plt.plot(B,A/np.max(A),c=colors[i])
plt.show()

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib as mpl
from skimage.transform import resize
from matplotlib.image import BboxImage
from matplotlib.transforms import Bbox, TransformedBbox
from sklearn.preprocessing import normalize
from scipy import ndimage
from skimage import morphology,filters
import pywt
import numpy as np
norm = mpl.colors.Normalize(vmin=0,vmax=1)
fig = plt.figure(figsize=(3,3),dpi=4000)
ax = fig.add_subplot(111)
artists = []
wavelet = 'db10'
keep_list=[4,5,6]
scale = int(2048*0.109)
BIGGER_SIZE = 12
plt.rc('font', size=BIGGER_SIZE, family='Arial')          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

for pos in md.image_table.Position.unique():
    if 'Pos' in pos:
        x,y = md.image_table[md.image_table.Position==pos].XY.iloc[0]
        if (x<-100)&(y<2750):
            im = np.flipud(md.stkread(Position=pos,Channel='DeepBlue',acq='nuc_highWellCapacity_6',Zindex=10)[:,:,0])
            thresh = np.percentile(im.ravel(),99)
            im[im>thresh]=thresh
            coeffs = pywt.wavedec2(im,wavelet)
            for i in range(0,len(coeffs)+1):
                if i in keep_list:
                    continue
                coeffs[-i] = tuple([np.zeros_like(v) for v in coeffs[-i]])
            Result = pywt.waverec2(coeffs,wavelet)
            thresh = np.percentile(Result.ravel(),99.9)
            Result[Result>thresh]=thresh
            Result[im<2**8.5]=np.nan
            if (x>-2500)&(x<-500)&(y>800)&(y<2000):
                im_bin = im>500
            else:
                im_bin = im>2000
            im_bin = ndimage.binary_dilation(im_bin,iterations=5)
            im_bin = morphology.remove_small_holes(im_bin, 2000)
            im_bin = morphology.remove_small_objects(im_bin, 10000)
            im_bin = morphology.erosion(im_bin)
            Result[im_bin==False]=np.nan
            Result = Result-np.nanpercentile(Result.ravel(),1)
            Result = Result/np.nanpercentile(Result.ravel(),99)
            bb = Bbox.from_bounds(x,y,scale,scale)
            bb2 = TransformedBbox(bb,ax.transData)
            bbox_image = BboxImage(bb2,
                                norm = norm,
                                origin=None,
                                clip_on=False,
                                cmap='Greys')
            bbox_image.set_data(Result)
            ax.add_artist(bbox_image)
ax.set_xlim(-3200,150)
ax.set_ylim(0,3000)
ax.set_xticks([])
ax.set_yticks([])
# plt.xlabel('X Stage Coordinate (um)')
# plt.ylabel('Y Stage Coordinate (um)')
#plt.suptitle('Wounded Cornea')
plt.axis('off')
plt.show()
fig.savefig('/home/zach/Desktop/Wounded_Cornea_no_axis')

In [ ]:
% matplotlib inline
norm = mpl.colors.Normalize(vmin=0,vmax=1)
BIGGER_SIZE = 12
plt.rc('font', size=BIGGER_SIZE, family='Arial') # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
temp_df = spotcalls[(spotcalls.ave>1000)&temp_mask]
x_offset = int(2048*0.109)
y_offset = 0
gene_names = ['Tnfr21','Casp9','Lrp5']
gene_list = ['Tnfrsf21','Casp9','Lrp5']
colors = ['b','r','g']
fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(414) 
for i,gene in enumerate(gene_list):
    temp = temp_df[(temp_df.gene==gene)]
    x = (temp.CoordX+(x_offset-np.min(X)))/0.109
    AB = np.histogram(x,bins=30,range=[1000,(2048*4)-400])
    A = AB[0]
    B = AB[1][0:-1]
    plt.plot(B,A/np.max(A),c=colors[i],label=gene)
    plt.xlim([800,(2048*4)-600])
    #plt.legend(loc='upper right',ncol=3)
    #plt.ylabel('Expression')
ax.set_yticks([0,1])
ax.yaxis.tick_right()
for i,gene in enumerate(gene_list):
    temp = temp_df[(temp_df.gene==gene)]
    if len(temp)<500:
        continue
    loc = int('41'+str(i+1))
    ax = fig.add_subplot(loc) 
    plt.imshow(np.flipud(Stitched[:,0:-400]),cmap='Greys',norm=norm, aspect="auto")
    x = (temp.CoordX+(x_offset-np.min(X)))/0.109
    y = (temp.CoordY+(y_offset-np.min(Y)))/0.109
    z = temp.z
    plt.scatter(x,y,c=colors[i],s=1,alpha=0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.yaxis.set_label_position('right')
    plt.ylabel(gene_names[i])
    plt.xlim([1000,(2048*4)-400])
    plt.ylim([0,2048])
    #plt.axis('off')
plt.show()
fig.savefig('/home/zach/Desktop/Wounded_Cornea_spots_zoom')

In [ ]:
md.image_table[md.image_table.Position=='Pos62']

In [ ]:
temp_df = spotcalls[(spotcalls.ave>1000)&
                    (spotcalls.posname=='Pos62')&
                    (spotcalls.CoordY>1500)&
                    (spotcalls.CoordY<1550)&
                    (spotcalls.CoordX>-1950)&
                    (spotcalls.CoordX<-1900)]
import matplotlib.pyplot as plt
%matplotlib notebook
plt.scatter(temp_df.CoordX,temp_df.CoordY,c=temp_df.cword_idx)
plt.show()

In [ ]:
import numpy as np
np.stack(temp_df.centroid)
import matplotlib.pyplot as plt
%matplotlib notebook
plt.scatter(np.stack(temp_df.centroid)[:,1],np.stack(temp_df.centroid)[:,0],c=temp_df.cword_idx)
plt.show()

In [ ]:
xy = np.stack(spotcalls.centroid)
x = xy[:,1]
y = xy[:,0]
temp_df = spotcalls[(spotcalls.ave>1000)&
                    (spotcalls.posname=='Pos62')&
                    (y>1625)&
                    (y<1640)&
                    (x>1400)&
                    (x<1420)]
%matplotlib notebook
plt.scatter(np.stack(temp_df.centroid)[:,1],np.stack(temp_df.centroid)[:,0],c=temp_df.ave)
plt.show()

In [ ]:
%matplotlib notebook
plt.scatter(np.stack(temp_df.centroid)[:,1],np.stack(temp_df.centroid)[:,0],c=temp_df.ave)
plt.show()